In [0]:
import os
import cv2
import json
from tqdm import tqdm, trange
import glob
import shutil as sh
import numpy as np
import random

# Retrieval of external Files

Download [JAAD](http://data.nvision2.eecs.yorku.ca/JAAD_dataset/) video files

In [0]:
datpath = '/content/JAAD'
if not os.path.exists(datpath):
  os.makedirs(datpath)
os.chdir(datpath)

In [0]:
!wget http://data.nvision2.eecs.yorku.ca/JAAD_dataset/data/JAAD_clips.zip

--2019-01-06 22:15:14--  http://data.nvision2.eecs.yorku.ca/JAAD_dataset/data/JAAD_clips.zip
Resolving data.nvision2.eecs.yorku.ca (data.nvision2.eecs.yorku.ca)... 130.63.92.182
Connecting to data.nvision2.eecs.yorku.ca (data.nvision2.eecs.yorku.ca)|130.63.92.182|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3084055074 (2.9G) [application/zip]
Saving to: ‘JAAD_clips.zip’

JAAD_clips.zip      100%[===================>]   2.87G  15.1MB/s    in 3m 36s  

2019-01-06 22:18:51 (13.6 MB/s) - ‘JAAD_clips.zip’ saved [3084055074/3084055074]



In [0]:
!unzip JAAD_clips

Archive:  JAAD_clips.zip
   creating: JAAD_clips/
  inflating: JAAD_clips/video_0286.mp4  
  inflating: JAAD_clips/video_0006.mp4  
  inflating: JAAD_clips/video_0109.mp4  
  inflating: JAAD_clips/video_0130.mp4  
  inflating: JAAD_clips/video_0210.mp4  
  inflating: JAAD_clips/video_0320.mp4  
  inflating: JAAD_clips/video_0162.mp4  
  inflating: JAAD_clips/video_0205.mp4  
  inflating: JAAD_clips/video_0046.mp4  
  inflating: JAAD_clips/video_0216.mp4  
  inflating: JAAD_clips/video_0315.mp4  
  inflating: JAAD_clips/video_0055.mp4  
  inflating: JAAD_clips/video_0163.mp4  
  inflating: JAAD_clips/video_0290.mp4  
  inflating: JAAD_clips/video_0057.mp4  
  inflating: JAAD_clips/video_0040.mp4  
  inflating: JAAD_clips/video_0310.mp4  
  inflating: JAAD_clips/video_0265.mp4  
  inflating: JAAD_clips/video_0131.mp4  
  inflating: JAAD_clips/video_0059.mp4  
  inflating: JAAD_clips/video_0023.mp4  
  inflating: JAAD_clips/video_0053.mp4  
  inflating: JAAD_clips/video_0191.mp4  
  infla

In [0]:
# delete zip files
!find . -type f -iname \*.zip -delete

Get bounding box data

In [0]:
!git clone https://github.com/ykotseruba/JAAD_pedestrian.git

Cloning into 'JAAD_pedestrian'...
remote: Enumerating objects: 2478, done.
remote: Total 2478 (delta 0), reused 0 (delta 0), pack-reused 2478
Receiving objects: 100% (2478/2478), 15.55 MiB | 7.00 MiB/s, done.
Resolving deltas: 100% (1962/1962), done.
Checking out files: 100% (1044/1044), done.


In [0]:
os.chdir('/content/')

In [0]:
!rm -rf cir-opencv
!git clone https://gitlab.com/emanuelb405/cir-opencv.git

Cloning into 'cir-opencv'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 205 (delta 57), reused 164 (delta 38)
Receiving objects: 100% (205/205), 125.34 MiB | 10.40 MiB/s, done.
Resolving deltas: 100% (57/57), done.


# Conversion

1.   \*.vbb bounding boxes to \*.xml
2.   \*.mp4 video files into single frame images



In [0]:
os.chdir('/content/')

Get data extractor (Python function based on this [repository](https://github.com/Ashwini-Analytics/Pedestrian-Detection-using-Darkflow) but adapted to our special needs)

In [0]:
!cp cir-opencv/JAAD/vbb2voc.py /content/

In [0]:
import vbb2voc

Define folder structure where dataset can be found and where results should be stored

In [0]:
vbb_in = '/content/JAAD/JAAD_pedestrian/vbb_part'
vid_in = '/content/JAAD/JAAD_clips'
dir_out = '/content/JAAD/extractedXML/'
vbb_out = os.path.join(dir_out, 'annot') 
vid_out = os.path.join(dir_out, 'image')

Split data into multiple sets (due to increased algorithm stability, Google Colab sometimes crashed when converting everything at once)

In [0]:
ext = {'vbb':'*.vbb', 'vid':'*.mp4'}
path = {'vbb':vbb_in, 'vid':vid_in}

batch_size = 10

for dtype in ['vbb','vid']:
  print('processing {} files'.format(dtype))
  
  files = glob.glob(os.path.join(path[dtype], ext[dtype]))
  files.sort(key = lambda x: int(os.path.splitext(os.path.split(x)[1])[0][-4:]))
  
  num_batches = int(np.ceil(len(files) / batch_size))
  num_data= len(files)
  
  for batch in trange(num_batches, desc='batch'):
    batch_path = os.path.join(path[dtype], 'set{:02d}'.format(batch))
    if not os.path.exists(batch_path):
      os.makedirs(batch_path)
    
    fSlice_ini = batch * batch_size
    fSlice_end = fSlice_ini + batch_size
    for file in tqdm(files[fSlice_ini:fSlice_end], desc='files'):
      filename = os.path.split(file)[1]
      dst = os.path.join(batch_path, filename)
      sh.move(file, dst)


files:   0%|          | 0/10 [00:00<?, ?it/s]


processing vbb files


files:   0%|          | 0/10 [00:00<?, ?it/s]

processing vid files



batch: 100%|██████████| 35/35 [00:00<00:00, 72.43it/s]


Convert files into required format

In [0]:
sets = next(os.walk(vbb_in))[1]
sets.sort(key = lambda x: int(x[-2:]))
for dset in sets:
  print('\n processing {}'.format(dset))
  vbb_in_set = os.path.join(vbb_in, dset)
  vid_in_set = os.path.join(vid_in, dset)
  vbb_out_set = os.path.join(vbb_out, dset)
  vid_out_set = os.path.join(vid_out, dset)
  vbb2voc.parse_anno_file(
      vbb_inputdir=vbb_in_set, 
      vid_inputdir=vid_in_set, 
      vbb_outputdir=vbb_out_set, 
      vid_outputdir=vid_out_set, 
      person_types=['pedestrian','ped'], 
      dictdat={'database': 'JAAD_clips', 'annotation': 'JAAD_pedestrian', 'image': 'JAAD', 'url': 'http://data.nvision2.eecs.yorku.ca/JAAD_dataset/'})

vbb file:   0%|          | 0/10 [00:00<?, ?it/s]


 processing set00



vbb file:   0%|          | 0/10 [00:00<?, ?it/s]


 processing set01



vbb file:   0%|          | 0/10 [00:00<?, ?it/s]

...


vbb file:   0%|          | 0/10 [00:00<?, ?it/s]


 processing set33



vbb file:   0%|          | 0/6 [00:00<?, ?it/s]


 processing set34



vbb file: 100%|██████████| 6/6 [00:33<00:00,  6.52s/it]


Split data into training and test data <br> Data is split up randomly but a random seed is provided to make collaboration possible (everyone is working with the same set of data)

In [0]:
# percentage of training data (remaining are for testing)
ratio = 0.9

In [0]:
# always generate same random permutations
np.random.seed(0)

In [0]:
# get total number of samples
tot_num_data = 0
subdirs = os.listdir(vbb_out)
for subdir in subdirs:
  sub_path = os.path.join(vbb_out, subdir)
  tot_num_data += len(os.listdir(sub_path))
print(tot_num_data)

75057


In [0]:
!rm -rf /content/JAAD/train
!rm -rf /content/JAAD/test

In [0]:
outpath = {'vbb':vbb_out, 'vid':vid_out}
trainpath = {'train': '/content/JAAD/train', 'test': '/content/JAAD/test'}

perm_files = []

for dtype in ['vbb','vid']:
  print('\n processing {} files'.format(dtype))
  processed = 0
  train = True
  
  subdirs = os.listdir(outpath[dtype])
  subdirs.sort()
  
  if dtype == 'vbb':
    perm_subdir = np.random.permutation(len(subdirs))
  subdirs = [x for _,x in sorted(zip(perm_subdir,subdirs))]  
    
  for idx, subdir in enumerate(subdirs):
    print('\n processing directory {}'.format(subdir))
    
    subdir_path = os.path.join(outpath[dtype], subdir)
    if os.path.isdir(subdir_path):
      files = os.listdir(subdir_path)
      files.sort()
      
      if dtype == 'vbb':
        perm_files.append(np.random.permutation(len(files)))
      files = [x for _,x in sorted(zip(perm_files[idx],files))]
      
      for file in tqdm(files):
        file_path = os.path.join(subdir_path, file)
        if train == True:
          dst = os.path.join(trainpath['train'], dtype)
        else:
          dst = os.path.join(trainpath['test'], dtype)
        
        if not os.path.exists(dst):
            os.makedirs(dst)
        
        file_dst = os.path.join(dst, file)
        
        sh.copyfile(file_path, file_dst)
        processed += 1
        train = False if processed > ratio*tot_num_data else True 

  0%|          | 0/2280 [00:00<?, ?it/s]


 processing vbb files

 processing directory set34

 processing directory set22


  5%|▍         | 106/2279 [00:00<00:02, 1055.70it/s]


 processing directory set07


 30%|███       | 652/2172 [00:00<00:00, 6512.24it/s]

...

  1%|          | 19/1847 [00:00<00:09, 185.98it/s]


 processing directory set32


  1%|          | 17/3048 [00:00<00:18, 167.92it/s]


 processing directory set11


100%|██████████| 3048/3048 [00:22<00:00, 223.91it/s]


Due to moving the files after converting, the folder entry (link to associated image) needs to be updated in the xml files

In [0]:
for dset in ['train', 'test']:
  print('\n processing {} set'.format(dset))
  filespath = os.path.join(trainpath[dset], 'vbb')
  newpath = os.path.join(trainpath[dset], 'vid')
  
  files = os.listdir(filespath)
  for file in tqdm(files):
    if 'xml' in os.path.splitext(file)[1]:
      filepath = os.path.join(filespath, file)
      with open(filepath, 'r') as rf:
        data = rf.readlines()
      
      data[1] = '<folder>' + newpath + '</folder>' + '\n'
      
      with open(filepath, 'w') as wf:
        wf.writelines(data)

  0%|          | 0/67552 [00:00<?, ?it/s]


 processing train set


  1%|▏         | 103/7505 [00:00<00:07, 1022.42it/s]


 processing test set


100%|██████████| 7505/7505 [00:06<00:00, 1155.87it/s]
